In [1]:
import pandas as pd
import numpy as np
import sqlalchemy
import psycopg2

In [2]:
players_data = pd.read_csv("C:/Users/ASUS/Desktop/players_integration.csv")

In [4]:
import pandas as pd
import numpy as np

# Load data
df = players_data.copy()

# --- SPLIT GOALKEEPERS VS OUTFIELD ---
mask_gk = df['main_pos'].str.contains('Goalkeeper', case=False, na=False) | df['Pos'].str.contains('GK', case=False, na=False)
df_gk = df[mask_gk].copy()
df_outfield = df[~mask_gk].copy()

print(f"Goalkeepers found: {len(df_gk)}")
print(f"Outfield players found: {len(df_outfield)}")

# --- DEFINE CLEANING CONFIGURATION ---

# 1. Metadata to Drop 
cols_metadata_drop = ['Born', 'Pos', 'Positions', 'Roles']

# 2. Team Bias Metrics
cols_team_bias_drop = [
    'W', 'D', 'L', 'PPM_Team.Success', 'onG_Team.Success', 'onGA_Team.Success',
    'plus_per__minus__Team.Success', 'plus_per__minus_90_Team.Success',
    'On_minus_Off_Team.Success', 'onxG_Team.Success..xG.',
    'onxGA_Team.Success..xG', 'On_minus_Off_Team.Success..xG',
    'xGplus_per__minus__Team.Success..xG', 'xGplus_per__minus_90_Team.Success..xG'
]

# 3. GK Specific Keywords 
gk_keywords = ['_GK', 'Save_', 'GA', 'SoTA', 'CS', 'PKsv', 'PKA', 'Sweeper']

# --- PROCESS OUTFIELD PLAYERS ---
# Drop Metadata + Team Bias + GK Stats
drop_outfield = cols_metadata_drop + cols_team_bias_drop + [c for c in df_outfield.columns if any(k in c for k in gk_keywords) and 'Att' not in c]
df_outfield_clean = df_outfield.drop(columns=drop_outfield, errors='ignore')

# --- PROCESS GOALKEEPERS ---
# Drop Metadata + Team Bias ONLY (Keep GK Stats)
drop_gk = cols_metadata_drop + cols_team_bias_drop
df_gk_clean = df_gk.drop(columns=drop_gk, errors='ignore')

print("Split and Drop complete.")

Goalkeepers found: 1115
Outfield players found: 9361
Split and Drop complete.


In [7]:
def convert_to_per90(df_input, is_gk=False):
    df_temp = df_input.copy()
    
    # Filter for valid minutes
    df_temp = df_temp[df_temp['Mins_Per_90'] > 0]

    # Standard Outfield Raw Stats
    raw_stat_cols = [
        'Cmp_Total', 'Att_Total', 'TotDist_Total', 'PrgDist_Total',
        'Cmp_Short', 'Att_Short', 'Cmp_Medium', 'Att_Medium', 'Cmp_Long', 'Att_Long',
        'Ast_Standard', 'KP', 'Final_Third', 'PPA', 'CrsPA', 'PrgP',
        'Touches_Touches', 'Def Pen_Touches', 'Def 3rd_Touches', 'Mid 3rd_Touches', 
        'Att 3rd_Touches', 'Att Pen_Touches', 'Live_Touches',
        'Att_Take', 'Succ_Take', 'Tkld_Take',
        'Carries_Carries', 'TotDist_Carries', 'PrgDist_Carries', 'PrgC_Carries',
        'Rec_Receiving', 'PrgR_Receiving',
        'Gls_Standard', 'Sh_Standard', 'SoT_Standard',
        'Tkl_Tackles', 'TklW_Tackles', 'Blocks_Blocks', 'Int_Def', 'Clr', 'Recov'
    ]

    # Add GK specific raw stats if this is the GK dataframe
    if is_gk:
        gk_cols = [c for c in df_temp.columns if any(k in c for k in gk_keywords)]
        # Add them to the list to be converted
        raw_stat_cols.extend(gk_cols)

    # Execute Conversion
    for col in raw_stat_cols:
        if col in df_temp.columns:
            # Check if numeric before dividing
            if pd.api.types.is_numeric_dtype(df_temp[col]):
                df_temp[col + '_Per90'] = df_temp[col] / df_temp['Mins_Per_90']
                df_temp.drop(columns=[col], inplace=True)
    
    
    # Fill categorical NaNs
    cat_cols = ['secondary_pos_1', 'secondary_pos_2', 'role_1', 'role_2', 'role_3', 'role_4']
    for col in cat_cols:
        if col in df_temp.columns:
            df_temp[col] = df_temp[col].fillna("None")
            
    return df_temp

# Apply to both
df_outfield_final = convert_to_per90(df_outfield_clean, is_gk=False)
df_gk_final = convert_to_per90(df_gk_clean, is_gk=True)

# --- STEP 3: AGGREGATE SEASONAL HISTORY (WEIGHTED) ---

def apply_season_weighting(df_input):
    print(f"Aggregating history for {len(df_input)} rows...")
    
    # 1. Define Metadata Columns (Columns to KEEP from the LATEST season)
    protected_cols = [
        'Player', 'Team', 'Nation', 'Pos', 'main_pos', 'Url',
        'secondary_pos_1', 'secondary_pos_2',
        'role_1', 'role_2', 'role_3', 'role_4',
        'Preferred foot', 'Season_End_Year', 'Age',
        'Mins_Per_90', 'Min_Playing.Time', 'Starts_Starts', 'Subs_Subs', 'unSub_Subs'
    ]
    
    
    # 2. Identify Stat Columns (The ones we will average)
    stat_cols = [
        c for c in df_input.columns 
        if pd.api.types.is_numeric_dtype(df_input[c]) 
        and c not in protected_cols
    ]

    DECAY_FACTOR = 0.75  # 2024=1.0, 2023=0.75, 2022=0.56...

    # 3. Define the aggregation logic
    def weighted_avg(group):
        # Sort: Newest season first
        group = group.sort_values('Season_End_Year', ascending=False)
        
        # Take the most recent metadata (Team, Age, Roles, etc.)
        composite_player = group.iloc[0].copy()
        
        # If only 1 season, we are done
        if len(group) == 1:
            return composite_player
        
        # Calculate Weights based on how old the season is
        latest_year = group['Season_End_Year'].max()
        years_ago = latest_year - group['Season_End_Year']
        weights = DECAY_FACTOR ** years_ago
        
        # Calculate Weighted Average for every stat column
        for col in stat_cols:
            # Only average if the column has data (not all NaNs)
            if group[col].notna().any():
                # Get values and valid weights
                vals = group[col]
                # Formula: Sum(Value * Weight) / Sum(Weights)
                # Fill NaNs with 0 (here we assume 0 for missing stats in a season)
                safe_vals = vals.fillna(0)
                composite_player[col] = np.average(safe_vals, weights=weights)
                
        return composite_player

    # 4. Apply GroupBy
    # This collapses the dataframe: 10k rows -> 3k unique players
    df_weighted = df_input.groupby('Player', group_keys=False).apply(weighted_avg)
    
    print(f"Result: {len(df_weighted)} unique player profiles.")
    return df_weighted

# --- EXECUTION ---

# Apply to Outfielders
print("--- Processing Outfielders ---")
df_outfield_aggregated = apply_season_weighting(df_outfield_final)

# Apply to Goalkeepers
print("--- Processing Goalkeepers ---")
df_gk_aggregated = apply_season_weighting(df_gk_final)


print(f"Outfield Cols: {df_outfield_final.shape[1]}")
print(f"GK Cols: {df_gk_final.shape[1]}")

--- Processing Outfielders ---
Aggregating history for 8372 rows...


C:\Users\ASUS\AppData\Local\Temp\ipykernel_22624\4199281315.py:118: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_weighted = df_input.groupby('Player', group_keys=False).apply(weighted_avg)


Result: 2543 unique player profiles.
--- Processing Goalkeepers ---
Aggregating history for 704 rows...
Result: 242 unique player profiles.
Outfield Cols: 200
GK Cols: 217


C:\Users\ASUS\AppData\Local\Temp\ipykernel_22624\4199281315.py:118: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_weighted = df_input.groupby('Player', group_keys=False).apply(weighted_avg)


In [8]:
from sklearn.preprocessing import MinMaxScaler

def create_embeddings(df_input, suffix=""):
    df_proc = df_input.copy()
    
    # 1. Fill NaNs
    df_proc.fillna(0, inplace=True)
    
    # 2. Encode Roles
    role_cols = ['role_1', 'role_2', 'role_3', 'role_4']
    all_roles = set()
    for col in role_cols:
        if col in df_proc.columns:
            unique_vals = df_proc[col].unique()
            all_roles.update(unique_vals)
            
    for role in all_roles:
        if role == 0 or role == "" or role == "None": continue
        df_proc[f"Role_{role}"] = df_proc[role_cols].apply(lambda x: 1 if role in x.values else 0, axis=1)

    # 3. Encode Preferred Foot 
    if 'Preferred foot' in df_proc.columns:
        df_proc = pd.get_dummies(df_proc, columns=['Preferred foot'], prefix='Foot')
        
    # 4. Scale Numeric Metrics
    # Exclude metadata and binary columns
    time_cols = ['Mins_Per_90', 'Min_Playing.Time', 'Starts_Starts', 'Subs_Subs', 'unSub_Subs']
    exclude = ['Player', 'Team', 'Nation', 'Pos', 'main_pos', 'secondary_pos_1', 'secondary_pos_2', 'Url', 'Age', 'Season_End_Year'] + role_cols + time_cols

    # Select numeric columns
    numeric_cols = df_proc.select_dtypes(include=['float64', 'int64', 'int32']).columns

    # Filter out the excluded columns
    numeric_cols = [c for c in numeric_cols if c not in exclude and not c.startswith('Role_') and not c.startswith('Foot_')]

    scaler = MinMaxScaler()
    df_proc[numeric_cols] = scaler.fit_transform(df_proc[numeric_cols])
    
    scaler = MinMaxScaler()
    df_proc[numeric_cols] = scaler.fit_transform(df_proc[numeric_cols])
    
    # 5. Create Vector
    # Ensure all features are float
    feature_cols = list(numeric_cols) + [c for c in df_proc.columns if c.startswith('Role_') or c.startswith('Foot_')]
    df_proc[feature_cols] = df_proc[feature_cols].astype(float)
    
    def get_vec(row):
        return row[feature_cols].values.tolist()

    df_proc[f'stats_embedding_{suffix}'] = df_proc.apply(get_vec, axis=1)
    
    return df_proc

# Calculate weighted averages (Combines seasons)
df_outfield_aggregated = apply_season_weighting(df_outfield_final)
df_gk_aggregated = apply_season_weighting(df_gk_final)

# Vectorize the AGGREGATED data
df_outfield_vectorized = create_embeddings(df_outfield_aggregated, suffix="outfield")
df_gk_vectorized = create_embeddings(df_gk_aggregated, suffix="gk")

# Check results
print("Example Outfield Vector Length:", len(df_outfield_vectorized.iloc[0]['stats_embedding_outfield']))
if not df_gk_vectorized.empty:
    print("Example GK Vector Length:", len(df_gk_vectorized.iloc[0]['stats_embedding_gk']))
    
# Save
df_outfield_vectorized.to_csv('scout_outfield.csv', index=False)
df_gk_vectorized.to_csv('scout_gk.csv', index=False)

Aggregating history for 8372 rows...


C:\Users\ASUS\AppData\Local\Temp\ipykernel_22624\4199281315.py:118: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_weighted = df_input.groupby('Player', group_keys=False).apply(weighted_avg)


Result: 2543 unique player profiles.
Aggregating history for 704 rows...


C:\Users\ASUS\AppData\Local\Temp\ipykernel_22624\4199281315.py:118: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_weighted = df_input.groupby('Player', group_keys=False).apply(weighted_avg)


Result: 242 unique player profiles.


C:\Users\ASUS\AppData\Local\Temp\ipykernel_22624\3069555990.py:50: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_proc[f'stats_embedding_{suffix}'] = df_proc.apply(get_vec, axis=1)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_22624\3069555990.py:50: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_proc[f'stats_embedding_{suffix}'] = df_proc.apply(get_vec, axis=1)


Example Outfield Vector Length: 212
Example GK Vector Length: 216
